<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

### Step 1: Generate COCO Annotations from PNG Masks

We assume that your preannotated masks are stored in the `dataset` folder in PNG format. Here's how to convert them into COCO format so that they can be uploaded to CVAT:

```python
import json
import numpy as np
from PIL import Image
import os
from pycocotools import mask as mask_utils

def masks_to_coco(mask_folder, mask_filenames, task_name, suffix, categories):
    output_coco_file = f"{task_name}{suffix}.json"
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": categories,
        "info": {"description": f"Dataset for {task_name} with segmentation masks"},
        "licenses": [],
    }

    annotation_id = 1
    image_id_counter = 1

    for mask_file in mask_filenames:
        mask_path = os.path.join(mask_folder, mask_file)
        mask = np.array(Image.open(mask_path))
        height, width = mask.shape[:2]

        coco_format["images"].append({
            "id": image_id_counter,
            "file_name": mask_file,
            "width": width,
            "height": height,
        })

        for category_id in np.unique(mask):
            if category_id == 0:
                continue

            binary_mask = (mask == category_id).astype(np.uint8)
            rle = mask_utils.encode(np.asfortranarray(binary_mask))
            rle['counts'] = rle['counts'].decode('utf-8')

            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id_counter,
                "category_id": int(category_id),
                "segmentation": rle,
                "area": float(np.sum(binary_mask)),
                "bbox": mask_utils.toBbox(rle).tolist(),
                "iscrowd": 0,
            })

            annotation_id += 1

        image_id_counter += 1

    with open(output_coco_file, 'w') as f:
        json.dump(coco_format, f, indent=4)

    return output_coco_file
```

This code will read the masks from the `mask_folder` and turn them into COCO format. Make sure you adjust the `mask_folder` and category list appropriately for your setup.

### Step 2: Upload Preannotations to CVAT

After generating the COCO annotations, we will upload them to the task in CVAT.

1. **Open the CVAT web app**: Navigate to `http://127.0.0.1:8080` in your browser.
2. **Login to CVAT**: Use your username (`django`) and password (`A1b2c345`).
3. **Select the Project**: Click on the `SurgVU` project, which is where the tasks are defined.
4. **Open the Task**: Select the task named `Iteration_1`.
5. **Upload the COCO file**: Navigate to the task, and upload the COCO file that was generated in Step 1.

To upload the annotations in CVAT, follow these steps:
1. Click on the task in the CVAT UI.
2. Use the "Upload Annotations" option and select the generated COCO file.

CVAT will now process the uploaded annotations, and the preannotated masks will appear in the task.

### Step 3: Review and Complete the Task in CVAT

Once the annotations have been uploaded, open the task in CVAT to review how the masks are rendered.

- **Review the Annotations**: Ensure that the preannotated masks are correctly applied to each image.
- **Edit and Complete the Task**: Make changes to any incorrect annotations or complete the labeling for images that were missed.
- **Save and Submit**: Once you're satisfied with the annotations, save the changes, complete the task, and submit it.

This will kickstart the fine-tuning process in MONAILabel. The server will detect that the task has been completed and automatically begin fine-tuning the model with the newly labeled data.

### What Happens Next

After the task is completed in CVAT and submitted:
1. MONAILabel will automatically detect the completion status of the task.
2. The server will trigger model fine-tuning with the newly generated labels.
3. You can monitor the fine-tuning process in MONAILabel and continue iterating through the active learning workflow.

In the next notebook, we'll explore how to manage the fine-tuning process and evaluate the newly trained models.